In [1]:
import os
import glob
import pandas as pd
from datetime import datetime
import locale
import re

In [71]:
class M8186:
    def __init__(self, fund_code, input_date, menu_code = '8186'):
        self.fund_code = fund_code
        self.menu_code = menu_code
        self.input_date = input_date
        self.df_ref = self.get_df_ref()
        self.get_df = self.get_df_filter()
        
        self.fund_name =self.get_fund_name()
        self.manager = self.get_manager()
        self.inception_date = self.get_inception_date() 
        self.inception_price = self.get_inception_price()
        self.net_asset_value = self.get_net_asset_value()
        self.reference_price = self.get_reference_price()
        self.cum_return = self.get_cum_return()

    def open_df_raw(self):
        lst = scan_files_including_regex('./캡스톤데이터', f'menu{self.menu_code}-code{self.fund_code}')
        lst= sorted(lst, reverse=True)
        file_path = lst[0]
        df = pd.read_csv(file_path) 
        return df
    
    def get_df_ref(self):
        df = self.open_df_raw()
        df = df[['일자', '펀드명', '운용역', '설정일', '설정액', '순자산', '수정기준가']]
        df['수정기준가'] = df['수정기준가'].apply(lambda x: float(x.replace(',', '')) if isinstance(x, str) else x)
        return df

    def get_fund_name(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['펀드명']]      
        fund_name = df['펀드명'].iloc[0] 
        return fund_name    
    
    def get_manager(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['운용역']]      
        manager = df['운용역'].iloc[0] 
        return manager   
    
    def get_inception_date(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['설정일']]      
        inception_date = df['설정일'].iloc[0]    
        return inception_date
    
    def get_inception_price(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['설정액']]      
        inception_price = df['설정액'].iloc[0]    
        return inception_price   
    
    def get_net_asset_value(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['순자산']]      
        net_asset_value = df['순자산'].iloc[0]    
        return net_asset_value        

    def get_reference_price(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['수정기준가']]      
        reference_price = df['수정기준가'].iloc[0]    
        return reference_price        
    
    def get_cum_return(self):
        df = self.df_ref
        first_price = df['수정기준가'].iloc[0]
        df = df[df['일자']==self.input_date][['수정기준가']]
        last_price = df['수정기준가'].iloc[0]
        cum_return = round((last_price-first_price)/first_price * 100, 2)
        return cum_return
        
    def get_info(self):
        info_data = {
            '항목': ['펀드명', '운용역', '운용개시일', '계약금액', '순자산금액', '기준가격', '누적수익률'],
            '값': [self.fund_name, self.manager, self.inception_date, self.inception_price, 
                   self.net_asset_value, self.reference_price, self.cum_return]
        }

        info_df = pd.DataFrame(info_data)
        return info_df 
    
    def get_df_filter(self): 
        df_8186 = self.df_ref
        df_8186['일자'] = pd.to_datetime(df_8186['일자'])

        input_date = pd.to_datetime(self.input_date)

        # input_date 이전의 데이터만 필터링
        df_8186 = df_8186[df_8186['일자'] <= input_date]
        return df_8186

In [72]:
m8186 = M8186(fund_code='A00001', input_date='2023-09-30')
m8186.get_df

,일자,펀드명,운용역,설정일,설정액,순자산,수정기준가
0,2023-05-11,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","9,997,067,848",999.71
1,2023-05-12,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","9,986,847,317",998.68
2,2023-05-13,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","9,987,765,196",998.78
3,2023-05-14,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","9,988,259,026",998.83
4,2023-05-15,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","10,002,863,755",1000.29
...,...,...,...,...,...,...,...
138,2023-09-26,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","10,754,656,468",1075.47
139,2023-09-27,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","10,784,791,521",1078.48
140,2023-09-28,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","10,785,037,396",1078.50
141,2023-09-29,라이프 ESG형 1호,강대권,2023-05-11,"10,000,000,000","10,785,283,270",1078.53


In [86]:
m8186 = M8186(fund_code='A00001', input_date='2023-09-30')
m8186.get_info()

,항목,값
0,펀드명,라이프 ESG형 1호
1,운용역,강대권
2,운용개시일,2023-05-11
3,계약금액,"10,000,000,000"
4,순자산금액,"10,785,529,144"
5,기준가격,1078.55
6,누적수익률,7.89


In [88]:
def create_dataframe(career_summary = None, education_certifications = None):

    career_summary = (
    "한국투자밸류자산운용 주식운용본부(주식운용), 2007.12 ~ 2014.02\n"
    "유경PSG자산운용 CIO(주식운용), 2014.02 ~ 2021.03\n"
    "라이프자산운용 공동대표(주식운용), 2021.06 ~ 현재"
    )
    education_certifications = (
        "서울대 경제학 학/석사\n"
        "투자자산운용사\n"
        "부동산투자자산운용사"
    )

   # 멀티인덱스 생성
    columns = pd.MultiIndex.from_tuples(
        [('성명', ''), ('직위', ''), ('생년월일', ''), ('연락처', ''), ('주요경력', '전,현직장의 근무부서 및 기간, 재직시 담당 업무'), ('주요경력', '학위 및 업무관련 자격증 취득 현황')]
    )

    # 데이터프레임 생성
    df = pd.DataFrame(columns=columns) 

    # 첫 번째 행에 값을 입력, 주요경력 부분에는 변수 사용
    df.loc[0] = ["강대권", "대표이사", "1980.11.24", "02-6105-6832", career_summary, education_certifications]

    # 데이터프레임 전치하여 반환
    return df.T



In [90]:
Investment_Professional_df = create_dataframe()
Investment_Professional_df

0
성명                                                                              강대권
직위                                                                             대표이사
생년월일                                                                     1980.11.24
연락처                                                                    02-6105-6832
주요경력 전,현직장의 근무부서 및 기간, 재직시 담당 업무  한국투자밸류자산운용 주식운용본부(주식운용), 2007.12 ~ 2014.02\n유경...
     학위 및 업무관련 자격증 취득 현황                          서울대 경제학 학/석사\n투자자산운용사\n부동산투자자산운용사

In [74]:
class M2820:
    def __init__(self, fund_code, input_date, menu_code = '2820'):
        self.fund_code = fund_code
        self.input_date = input_date
        self.menu_code = menu_code
        self.df_ref = self.get_df_ref()
        self.get_df = self.get_df_filter()

    def open_df_raw(self):
        lst = scan_files_including_regex('./캡스톤데이터', f'menu{self.menu_code}-code{self.fund_code}')
        lst= sorted(lst, reverse=True)
        file_path = lst[0]
        df = pd.read_csv(file_path) 
        return df
    
    def get_df_ref(self):
        df = self.open_df_raw()
        df = df[['일자', '매매구분', '수수료', '취득액']]
        df['수수료'] = df['수수료'].str.replace(',', '').astype(float)
        df['취득액'] = df['취득액'].str.replace(',', '').astype(float)
        return df.groupby(['일자', '매매구분']).agg({'수수료': 'sum', '취득액': 'sum'}).reset_index()
    
    def get_df_filter(self): 
        df_2820 = self.df_ref
        df_2820['일자'] = pd.to_datetime(df_2820['일자'])

        input_date = pd.to_datetime(self.input_date)

        # input_date 이전의 데이터만 필터링
        df_2820 = df_2820[df_2820['일자'] <= input_date]
        return df_2820

In [75]:
m2820 = M2820(fund_code='A00001', input_date='2023-09-30')
m2820.get_df

,일자,매매구분,수수료,취득액
0,2023-05-11,주식장내매수,1343790.0,1.345400e+09
1,2023-05-12,주식장내매수,674830.0,6.757427e+08
2,2023-05-15,주식장내매도,23600.0,2.603107e+07
3,2023-05-15,주식장내매수,943200.0,9.445024e+08
4,2023-05-16,주식장내매도,33120.0,3.634371e+07
...,...,...,...,...
154,2023-09-26,ETF장내매입,104550.0,1.047136e+08
155,2023-09-26,주식장내매도,447300.0,4.734266e+08
156,2023-09-26,주식장내매수,233820.0,2.343317e+08
157,2023-09-27,주식장내매도,150840.0,1.690474e+08


In [84]:
class guideline_status:
    def __init__(self, fund_code, inpt_date):
        self.fund_code = fund_code
        self.inpt_date = inpt_date
        
    def merge_date(self):
        m2820 = M2820(fund_code=self.fund_code, input_date=self.inpt_date)
        m8186 = M8186(fund_code=self.fund_code, input_date=self.inpt_date)
        df_2820 = m2820.get_df
        df_8186 = m8186.get_df
        df_8186 = df_8186[['일자', '순자산']].copy()
        df_8186['순자산'] = df_8186['순자산'].str.replace(',', '').astype(float)
        merged_df = pd.merge(df_2820, df_8186, on='일자', how='inner')
        merged_df['일자'] = pd.to_datetime(merged_df['일자'])
        return merged_df
    
    def calculate_monthly_turnover(self, merged_df):
        
        monthly_turnovers = {}
        for period in merged_df['일자'].dt.to_period('M').unique():
            month_df = merged_df[merged_df['일자'].dt.to_period('M') == period]
            total_acquisition = month_df['취득액'].sum()  # 총 취득액 계산
            average_net_asset = month_df['순자산'].mean() * 2  # 평균 순자산의 두 배 계산
            monthly_turnovers[period.strftime('%Y-%m')] = total_acquisition / average_net_asset if average_net_asset else 0

        return monthly_turnovers

    def calculate_monthly_fee(self, merged_df):

        monthly_fees = {}
        for period in merged_df['일자'].dt.to_period('M').unique():
            month_df = merged_df[merged_df['일자'].dt.to_period('M') == period]
            fee = month_df['수수료'].sum()
            monthly_fees[period.strftime('%Y-%m')] = fee
        return monthly_fees

    def calculate_turnover_and_fee(self):
        merged_df = self.merge_date()
        monthly_turnovers = self.calculate_monthly_turnover(merged_df)
        monthly_fees = self.calculate_monthly_fee(merged_df)
        # 월간 값과 누적 값 계산
        last_turnover = list(monthly_turnovers.values())[-1] if monthly_turnovers else 0
        last_fee = list(monthly_fees.values())[-1] if monthly_fees else 0
        total_turnover = sum(monthly_turnovers.values())
        total_fee = sum(monthly_fees.values())

        # 소수점 둘째자리까지 반올림
        last_turnover = round(last_turnover * 100, 2)
        total_turnover = round(total_turnover * 100, 2)

        return last_turnover, total_turnover, last_fee, total_fee

    def get_df_performance(self):
        merged_df = self.merge_date()
        monthly_turnovers = self.calculate_monthly_turnover(merged_df)
        monthly_fees = self.calculate_monthly_fee(merged_df)

        # 성과 측정 지표 계산
        last_turnover, total_turnover, last_fee, total_fee = self.calculate_turnover_and_fee()

        turnover_rate = f"{last_turnover}/{total_turnover}"
        transaction_fee = f"{last_fee}/{total_fee}"
        no_records = '해당내역없음'
        performance_df = pd.DataFrame({
            'header': [no_records, no_records, turnover_rate, transaction_fee, no_records, no_records],
        }, index=['운용대상 제한', '편입비 제한', '매매회전율(월간/누적)', '매매수수료(월간/누적)', '종목손절매', '투자일임자산 및 고유자산과의 중복거래 내역'])        
        
        return performance_df

In [29]:
def scan_files_including_regex(file_folder, regex, option='path'):
    with os.scandir(file_folder) as files:
        lst = [file.name for file in files if re.findall(regex, file.name)]
    mapping = {
        'name': lst,
        'path': [os.path.join(file_folder, file_name) for file_name in lst]
    }
    return mapping[option]

In [85]:
gs = guideline_status(fund_code='A00001', inpt_date='2023-09-30')

gs.get_df_performance()

,header
운용대상 제한,해당내역없음
편입비 제한,해당내역없음
매매회전율(월간/누적),15.46/111.26
매매수수료(월간/누적),3413800.0/23251600.0
종목손절매,해당내역없음
투자일임자산 및 고유자산과의 중복거래 내역,해당내역없음


In [99]:
class Report_page_one:
    def __init__(self, fund_code, input_date):
        self.fund_code = fund_code
        self.input_date = input_date

    def get_combined_report(self):
        # M8186 클래스 인스턴스 생성 및 정보 추출
        m8186 = M8186(fund_code=self.fund_code, input_date=self.input_date)
        fund_info_df = m8186.get_info()

        # Investment_Professional_df 생성
        investment_professional_df = create_dataframe()

        # guideline_status 클래스 인스턴스 생성 및 성과 데이터 추출
        gs = guideline_status(fund_code=self.fund_code, inpt_date=self.input_date)
        performance_df = gs.get_df_performance()

        # 데이터프레임들을 리스트로 결합
        combined_df = [fund_info_df, investment_professional_df, performance_df]

        return combined_df
    

In [100]:
report = Report_page_one(fund_code='A00001', input_date='2023-09-30')
combined_report = report.get_combined_report()
combined_report

[      항목               값
 0    펀드명     라이프 ESG형 1호
 1    운용역             강대권
 2  운용개시일      2023-05-11
 3   계약금액  10,000,000,000
 4  순자산금액  10,785,529,144
 5   기준가격         1078.55
 6  누적수익률            7.89,
                                                                                   0
 성명                                                                              강대권
 직위                                                                             대표이사
 생년월일                                                                     1980.11.24
 연락처                                                                    02-6105-6832
 주요경력 전,현직장의 근무부서 및 기간, 재직시 담당 업무  한국투자밸류자산운용 주식운용본부(주식운용), 2007.12 ~ 2014.02\n유경...
      학위 및 업무관련 자격증 취득 현황                          서울대 경제학 학/석사\n투자자산운용사\n부동산투자자산운용사,
                                        header
 운용대상 제한                                해당내역없음
 편입비 제한                                 해당내역없음
 매매회전율(월간/누적)                     15.46/111.26
 매매수수료(